##### Lenet-5:
<img src="Lenet5/Lenet5.png">


In [4]:
# inference
import tensorflow as tf
import numpy as np
import os

IMAGE_SIZE = 28  # 输入维度大小
NUM_CHANNELS = 1  # 通道数
CONV1_SIZE = 5  # 卷积核维度
CONV1_KERNEL_NUM = 32  # 卷积核个数
CONV2_SIZE = 5
CONV2_KERNEL_NUM = 64
FC_SIZE = 512  # 全连接层节点
OUTPUT_SIZE = 10  # 输出节点（分类数目）

def create_placeholder():
    with tf.name_scope('input_data'):
        X = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS], name = 'X')
        Y = tf.placeholder(tf.float32, [None, OUTPUT_SIZE], name = 'Y')
#         image_shaped_input = tf.reshape(X, [-1, 28, 28, 1])
        tf.summary.image('X_images', X, 10)
    return X, Y

def get_weight(shape, regularizer):  # 权重初始化  # tf.get_variable('weight', shape, initializer = tf.truncated_normal_initializer(stddev=0.1))
    W = tf.Variable(tf.truncated_normal(shape, stddev=0.1), name = 'weight')
    if regularizer != None:
        tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(W))
    tf.summary.histogram('weight', W)
    return W

def get_biasis(shape):  # 偏置初始化
    b = tf.Variable(tf.zeros(shape), name = 'biasis')
    tf.summary.histogram('biasis', b)
    return b
    
def conv2d(X, W):  # 卷积操作，X: [N, H, W, C]
    return tf.nn.conv2d(X, W, strides = [1,1,1,1], padding = 'SAME')

def max_pool(X):  # 最大池化，2x2，步长2
    return tf.nn.max_pool(X, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

def reshaped(pool):  # 全连接层维度拉直
    pool_shape = pool.get_shape().as_list()
#     print(pool_shape)
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]  # reshaped = tf.contrib.layers.flatten(pool2)
    reshaped = tf.contrib.layers.flatten(pool)
#     reshaped = tf.reshape(pool, [pool_shape[0], nodes])
    return reshaped, nodes

def forward(X, train = True, regularizer = 0.0001):  # 前向传播，train决定训练时使用dropout    
    with tf.variable_scope('conv32_5x5'):  # 第一层
        conv1_w = get_weight([CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_KERNEL_NUM], regularizer)
        conv1_b = get_biasis([CONV1_KERNEL_NUM])
        conv1 = conv2d(X, conv1_w)
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_b))
        pool1 = max_pool(relu1)
        
    with tf.variable_scope('conv64_5x5'):  # 第二层
        conv2_w = get_weight([CONV2_SIZE,CONV2_SIZE,CONV1_KERNEL_NUM,CONV2_KERNEL_NUM], regularizer)
        conv2_b = get_biasis([CONV2_KERNEL_NUM])
        conv2 = conv2d(pool1, conv2_w)
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_b))
        pool2 = max_pool(relu2)
        
    with tf.variable_scope('fc_512'):  # 全连接层
        reshaped1, nodes = reshaped(pool2)
        fc1_w = get_weight([nodes, FC_SIZE], regularizer)
        fc1_b = get_biasis([FC_SIZE])
        fc1 = tf.nn.relu(tf.matmul(reshaped1, fc1_w) + fc1_b)
        if train == True: fc1 = tf.nn.dropout(fc1, 0.5)  # 训练时dropout
        
    with tf.variable_scope('output_10'):  # 输出全连接层
        o_w = get_weight([FC_SIZE, OUTPUT_SIZE], regularizer)
        o_b = get_biasis([OUTPUT_SIZE])
        y = tf.matmul(fc1, o_w) + o_b
    
    return y



In [9]:
#读取frecords并还原
#=======================================================================================
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

def read_and_decode(filename):  
    # 创建文件队列,不限读取的数量  
    filename_queue = tf.train.string_input_producer([filename])  
    # create a reader from file queue  
    reader = tf.TFRecordReader()  
    # reader从文件队列中读入一个序列化的样本  
    _, serialized_example = reader.read(filename_queue)  
    # get feature from serialized example  
    # 解析符号化的样本  
    features = tf.parse_single_example(  #按原保存的格式解析
        serialized_example,  
        features={  
            'label': tf.FixedLenFeature([], tf.int64),  
            'image_raw': tf.FixedLenFeature([], tf.string)  
        })  
    lab = features['label']  
    img = features['image_raw']  
    de_img = tf.decode_raw(img, tf.uint8) #图像解析为对应的像素数组
    image = tf.reshape(de_img, [28, 28, 1]) # 
    images = tf.cast(image, tf.float32) * (1. / 255)
    label = tf.cast(lab, tf.int32)
    labels = tf.cast(tf.one_hot(label, 10), tf.float32)
    return image, labels 

def get_tf(batch_size, Train=True):
    if Train:
        tf_path = tr_tf_path
    else:
        tf_path = te_tf_path
    img, label = read_and_decode(tf_path)
    img_batch, label_batch = tf.train.shuffle_batch([img,label],
                                                   batch_size=batch_size,
                                                   num_threads=4,
                                                   capacity=1000,
                                                   min_after_dequeue=600)
    img_batch = tf.cast(img_batch, tf.float32)
    return img_batch, label_batch

tr_tf_path, tr_img_path = 'G:/MNIST_data/train/train.tfrecords', 'G:/MNIST_data/train/'
te_tf_path, te_img_path = 'G:/MNIST_data/test/mnist_test.tfrecords', 'G:/MNIST_data/test/'
# batch = get_tf(256, Train=True)
# # batch = read_and_decode(tr_tf_path)
# with tf.Session() as sess:
# #     sess.run(tf.initialize_all_variables())
#     coord=tf.train.Coordinator()    
#     threads= tf.train.start_queue_runners(coord=coord)
#     for i in range(60000):
#         example, lab = sess.run(batch)
#         print(example.shape,lab.shape)
#         print(lab)
    
    
#     coord.request_stop()
#     coord.join(threads)

(?,)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer_2/input_producer_2_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer_2, input_producer_2/Identity/_45)]]


KeyboardInterrupt: 

In [6]:
from tensorflow.examples.tutorials.mnist import input_data

STEPS = 100  # 根据所有样本的代数停止
INTERATIONS = 10000  # 根据迭代次数停止训练
BATCH_SIZE = 512
LEARNING_RATE_BASE = 0.003
LEARNING_RATE_DECAY = 0.99
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = './Lenet5/model/'
MODEL_NAME = 'Lenet-5_model'

if not os.path.exists(MODEL_SAVE_PATH): os.mkdir(MODEL_SAVE_PATH)

# mnist = input_data.read_data_sets("G:/MNIST_data/", one_hot = True)
def backward():
    tf.reset_default_graph()
    X, Y = create_placeholder()
    y = forward(X, True)
    
    global_step = tf.Variable(0, trainable = False, name = 'train_step')
    learning_rate = tf.train.exponential_decay(  # 学习率指数衰减，
        LEARNING_RATE_BASE, 
        global_step, 
        60000 / BATCH_SIZE, 
        LEARNING_RATE_DECAY, 
        staircase = True)
    tf.summary.scalar('learning_rate', learning_rate)
    
    with tf.name_scope('optimizer'):
        with tf.name_scope('loss'):
            mean_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = Y))  
            # tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y, labels = tf.argmax(Y, 1))
            tf.add_to_collection('losses', mean_loss)  # loss = mean_loss + tf.add_n(tf.get_collection('losses'))
            loss = tf.add_n(tf.get_collection('losses'))
            tf.summary.scalar('mean_loss', mean_loss)
            tf.summary.scalar('loss_l2', loss)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)  # 滑动平均
        ema_op = ema.apply(tf.trainable_variables())
        with tf.control_dependencies([train_step, ema_op]):  # 训练时同步更新滑动平均值
            train_op = tf.no_op(name = 'train')
        
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')
        tf.summary.scalar('train', accuracy)
        
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # 允许显存增长，（显存不足）
    saver = tf.train.Saver()
    
    b_xs, b_ys = get_tf(BATCH_SIZE, Train=True)
    with tf.Session(config = config) as sess:
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(MODEL_SAVE_PATH, sess.graph)
        sess.run(tf.global_variables_initializer())
        
        coord=tf.train.Coordinator()    
        threads= tf.train.start_queue_runners(coord=coord)
        
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        
#         step = sess.run(global_step)
        n_batch = int(60000 / BATCH_SIZE)
        for epoch in range(STEPS): # range(step, STEPS)
            epoch_loss = 0
            epoch_acc = 0
            for i in range(n_batch):
#                 xs, ys = mnist.train.next_batch(BATCH_SIZE)
                xs, ys = sess.run([b_xs,b_ys])
#                 reshaped_xs = np.reshape(xs, [BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS])
                _, loss_value, acc, step, summary = sess.run([train_op, loss, accuracy, global_step, merged], feed_dict = {X:xs, Y:ys})
#                 print(step)
                epoch_loss += loss_value/n_batch
                epoch_acc += acc/n_batch
                writer.add_summary(summary, step)
                print('INFO: After %d training iteration(s), loss is %.5f, batch_train accuracy is %.4f' % (step, loss_value, acc))
                if step % 100 == 0:
                    saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME+ '.ckpt'), global_step = global_step)
                    print('%s-%d.ckpt saved!' % (MODEL_NAME, step))
                if step >= INTERATIONS: return
#                     print('INFO: After %d training iteration(s), loss is %.5f, batch_train accuracy is %.4f' % (step, loss_value, acc))
#             if epoch % 10 == 0:
            print('After %d training epoch(s)(X%d), loss is %.5f, train accuracy is %.4f' % (epoch+1, n_batch, epoch_loss, epoch_acc))
            saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME+'.ckpt-{:d}x{:d}').format(epoch+1, n_batch))
            print('%s-%dx%d saved!' % (MODEL_NAME, epoch+1, n_batch))
#             sess.run(global_step.assign(epoch))
#             print('%s-%d saved!' % (MODEL_NAME, epoch))
#             if step >= INTERATIONS: break
        coord.request_stop()
        coord.join(threads)
    
backward()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from ./Lenet5/model/Lenet-5_model.ckpt-200
INFO: After 201 training iteration(s), loss is 2.94561, batch_train accuracy is 0.1328
INFO: After 202 training iteration(s), loss is 2.96005, batch_train accuracy is 0.1035
INFO: After 203 training iteration(s), loss is 2.94954, batch_train accuracy is 0.1172
INFO: After 204 training iteration(s), loss is 2.96064, batch_train accuracy is 0.1191
INFO: After 205 training iteration(s), loss is 2.95829, batch_train accuracy is 0.1074
INFO: After 206 training iteration(s), loss is 2.96427, batch_train accuracy is 0.1250
INFO: After 207 training iteration(s), loss is 2.95305, batch_train accuracy is 0.1250
INFO: After 208 training iteration(s), loss is 2.94703, batch_train accuracy is 0.1309
INFO: After 209 training iterati

INFO: After 290 training iteration(s), loss is 2.93490, batch_train accuracy is 0.1465
INFO: After 291 training iteration(s), loss is 2.95068, batch_train accuracy is 0.1094
INFO: After 292 training iteration(s), loss is 2.94672, batch_train accuracy is 0.1230
INFO: After 293 training iteration(s), loss is 2.95667, batch_train accuracy is 0.1074
INFO: After 294 training iteration(s), loss is 2.94537, batch_train accuracy is 0.1270
INFO: After 295 training iteration(s), loss is 2.93512, batch_train accuracy is 0.1406
INFO: After 296 training iteration(s), loss is 2.93885, batch_train accuracy is 0.1328
INFO: After 297 training iteration(s), loss is 2.95442, batch_train accuracy is 0.1387
INFO: After 298 training iteration(s), loss is 2.94102, batch_train accuracy is 0.1406
INFO: After 299 training iteration(s), loss is 2.95054, batch_train accuracy is 0.1348
INFO: After 300 training iteration(s), loss is 2.93416, batch_train accuracy is 0.1562
Lenet-5_model-300.ckpt saved!
INFO: After 3

INFO: After 383 training iteration(s), loss is 2.92435, batch_train accuracy is 0.1445
INFO: After 384 training iteration(s), loss is 2.93753, batch_train accuracy is 0.1484
INFO: After 385 training iteration(s), loss is 2.93079, batch_train accuracy is 0.1309
INFO: After 386 training iteration(s), loss is 2.92974, batch_train accuracy is 0.1484
INFO: After 387 training iteration(s), loss is 2.93652, batch_train accuracy is 0.1641
INFO: After 388 training iteration(s), loss is 2.93959, batch_train accuracy is 0.1270
INFO: After 389 training iteration(s), loss is 2.93377, batch_train accuracy is 0.1465
INFO: After 390 training iteration(s), loss is 2.92820, batch_train accuracy is 0.1348
INFO: After 391 training iteration(s), loss is 2.93738, batch_train accuracy is 0.1367
INFO: After 392 training iteration(s), loss is 2.94219, batch_train accuracy is 0.1367
INFO: After 393 training iteration(s), loss is 2.92844, batch_train accuracy is 0.1328
INFO: After 394 training iteration(s), loss

INFO: After 476 training iteration(s), loss is 2.92194, batch_train accuracy is 0.1543
INFO: After 477 training iteration(s), loss is 2.91735, batch_train accuracy is 0.1738
INFO: After 478 training iteration(s), loss is 2.92920, batch_train accuracy is 0.1230
INFO: After 479 training iteration(s), loss is 2.91637, batch_train accuracy is 0.1699
INFO: After 480 training iteration(s), loss is 2.92649, batch_train accuracy is 0.1504
INFO: After 481 training iteration(s), loss is 2.91731, batch_train accuracy is 0.1582
INFO: After 482 training iteration(s), loss is 2.92077, batch_train accuracy is 0.1738
INFO: After 483 training iteration(s), loss is 2.91793, batch_train accuracy is 0.1660
INFO: After 484 training iteration(s), loss is 2.92634, batch_train accuracy is 0.1543
INFO: After 485 training iteration(s), loss is 2.92973, batch_train accuracy is 0.1660
INFO: After 486 training iteration(s), loss is 2.93679, batch_train accuracy is 0.1543
INFO: After 487 training iteration(s), loss

INFO: After 569 training iteration(s), loss is 2.91927, batch_train accuracy is 0.1445
INFO: After 570 training iteration(s), loss is 2.90414, batch_train accuracy is 0.1797
INFO: After 571 training iteration(s), loss is 2.90192, batch_train accuracy is 0.1582
INFO: After 572 training iteration(s), loss is 2.92109, batch_train accuracy is 0.1621
INFO: After 573 training iteration(s), loss is 2.89958, batch_train accuracy is 0.1855
INFO: After 574 training iteration(s), loss is 2.90131, batch_train accuracy is 0.1836
INFO: After 575 training iteration(s), loss is 2.91384, batch_train accuracy is 0.1582
INFO: After 576 training iteration(s), loss is 2.91368, batch_train accuracy is 0.1680
INFO: After 577 training iteration(s), loss is 2.90933, batch_train accuracy is 0.1895
INFO: After 578 training iteration(s), loss is 2.89977, batch_train accuracy is 0.2012
INFO: After 579 training iteration(s), loss is 2.90729, batch_train accuracy is 0.1816
INFO: After 580 training iteration(s), loss

...

INFO: After 1128 training iteration(s), loss is 2.71908, batch_train accuracy is 0.3730
INFO: After 1129 training iteration(s), loss is 2.72862, batch_train accuracy is 0.3691
INFO: After 1130 training iteration(s), loss is 2.72329, batch_train accuracy is 0.3145
INFO: After 1131 training iteration(s), loss is 2.71571, batch_train accuracy is 0.3594
INFO: After 1132 training iteration(s), loss is 2.70998, batch_train accuracy is 0.3594
INFO: After 1133 training iteration(s), loss is 2.73026, batch_train accuracy is 0.3203
INFO: After 1134 training iteration(s), loss is 2.75638, batch_train accuracy is 0.2910
INFO: After 1135 training iteration(s), loss is 2.70758, batch_train accuracy is 0.3262
INFO: After 1136 training iteration(s), loss is 2.73539, batch_train accuracy is 0.3164
After 8 training epoch(s)(X117), loss is 2.75698, train accuracy is 0.3179
Lenet-5_model-8x117 saved!
INFO: After 1137 training iteration(s), loss is 2.70834, batch_train accuracy is 0.3477
INFO: After 1138 t

INFO: After 1220 training iteration(s), loss is 2.63749, batch_train accuracy is 0.3945
INFO: After 1221 training iteration(s), loss is 2.65839, batch_train accuracy is 0.3633
INFO: After 1222 training iteration(s), loss is 2.66284, batch_train accuracy is 0.3555
INFO: After 1223 training iteration(s), loss is 2.66158, batch_train accuracy is 0.3887
INFO: After 1224 training iteration(s), loss is 2.67491, batch_train accuracy is 0.3438
INFO: After 1225 training iteration(s), loss is 2.66929, batch_train accuracy is 0.3730
INFO: After 1226 training iteration(s), loss is 2.68587, batch_train accuracy is 0.3633
INFO: After 1227 training iteration(s), loss is 2.68183, batch_train accuracy is 0.3574
INFO: After 1228 training iteration(s), loss is 2.64817, batch_train accuracy is 0.4043
INFO: After 1229 training iteration(s), loss is 2.65538, batch_train accuracy is 0.3574
INFO: After 1230 training iteration(s), loss is 2.61754, batch_train accuracy is 0.3945
INFO: After 1231 training iterat

INFO: After 1312 training iteration(s), loss is 2.56118, batch_train accuracy is 0.4062
INFO: After 1313 training iteration(s), loss is 2.58068, batch_train accuracy is 0.3887
INFO: After 1314 training iteration(s), loss is 2.56698, batch_train accuracy is 0.4082
INFO: After 1315 training iteration(s), loss is 2.56129, batch_train accuracy is 0.4141
INFO: After 1316 training iteration(s), loss is 2.57878, batch_train accuracy is 0.3984
INFO: After 1317 training iteration(s), loss is 2.54811, batch_train accuracy is 0.4297
INFO: After 1318 training iteration(s), loss is 2.56681, batch_train accuracy is 0.4316
INFO: After 1319 training iteration(s), loss is 2.53733, batch_train accuracy is 0.4219
INFO: After 1320 training iteration(s), loss is 2.55494, batch_train accuracy is 0.4180
INFO: After 1321 training iteration(s), loss is 2.53272, batch_train accuracy is 0.4395
INFO: After 1322 training iteration(s), loss is 2.52911, batch_train accuracy is 0.4141
INFO: After 1323 training iterat

INFO: After 1404 training iteration(s), loss is 2.46535, batch_train accuracy is 0.4277
INFO: After 1405 training iteration(s), loss is 2.48018, batch_train accuracy is 0.4258
INFO: After 1406 training iteration(s), loss is 2.47132, batch_train accuracy is 0.4297
INFO: After 1407 training iteration(s), loss is 2.44225, batch_train accuracy is 0.4551
INFO: After 1408 training iteration(s), loss is 2.43179, batch_train accuracy is 0.4941
INFO: After 1409 training iteration(s), loss is 2.44167, batch_train accuracy is 0.4688
INFO: After 1410 training iteration(s), loss is 2.47478, batch_train accuracy is 0.4316
INFO: After 1411 training iteration(s), loss is 2.47853, batch_train accuracy is 0.4121
INFO: After 1412 training iteration(s), loss is 2.49591, batch_train accuracy is 0.4473
INFO: After 1413 training iteration(s), loss is 2.45505, batch_train accuracy is 0.4473
INFO: After 1414 training iteration(s), loss is 2.42290, batch_train accuracy is 0.4492
INFO: After 1415 training iterat

KeyboardInterrupt: 

In [ ]:
# 评估准确率
import tensorflow as tf
import time
import numpy as np

TEST_INTERVAL_SECS = 5  # 间隔

def evaluation(mnist_image, mnist_label, num_examples):  # mnist.test.images, minst.test.num_examples
    with tf.Graph().as_default() as g:
        X = tf.placeholder(tf.float32, [None, 28, 28, 1])
        Y = tf.placeholder(tf.float32, [None, 10])
        y = forward(X, False, None)
        
        em = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)  # 滑动平均
        em_restore = em.variables_to_restore()
        saver = tf.train.Saver(em_restore)
        
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
#         while True:
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                steps = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                reshape_x = np.reshape(mnist_image, [num_examples, 28, 28, 1])
                acc_ = sess.run(accuracy, feed_dict = {X:reshape_x, Y:mnist_label})
                print('After %s training step(s), evaluation accuracy is %.4f' % (steps, acc_))
            else:
                print('No checkpoint file found!')
                return
#             time.sleep(TEST_INTERVAL_SECS)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("G:/MNIST_data/", one_hot = True)
evaluation(mnist.test.images, mnist.test.labels, mnist.test.num_examples)